<a href="https://colab.research.google.com/github/27ABH/AI-Data-Science-2025-26/blob/main/Next_Word_Predictor_Pt_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch

# Load model
print("Loading AI model...")
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

Loading AI model...


In [16]:
# Simple example
text = "Yesterday I learned how to skateboard but I injured my knee in the process"
print(f"\nStarting text: '{text}'")
print("\nGenerating word by word...\n")

# --- ADD THESE PARAMETERS ---
k = 3
temperature = 0.8  # Lower = more focused, Higher = more creative/random
repetition_penalty = 1.2  # Values > 1.0 penalize used words

input_ids = tokenizer.encode(text, return_tensors="pt")

for step in range(30):
    with torch.no_grad():
        outputs = model(input_ids)
        # 1. Apply Temperature to the raw logits
        next_token_logits = outputs.logits[0, -1, :] / temperature

        # 2. APPLY REPETITION PENALTY
        # We look at every token we've already generated
        for prev_token_id in set(input_ids[0].tolist()):
            # If the logit is positive, we make it smaller (divide)
            # If it's negative, we make it even more negative (multiply)
            if next_token_logits[prev_token_id] > 0:
                next_token_logits[prev_token_id] /= repetition_penalty
            else:
                next_token_logits[prev_token_id] *= repetition_penalty

        next_token_probs = torch.softmax(next_token_logits, dim=0)

    # 3. Top-K Sampling
    top_probs, top_indices = torch.topk(next_token_probs, k)
    sample_index = torch.multinomial(top_probs, num_samples=1)
    next_token_id = top_indices[sample_index].unsqueeze(0)

    input_ids = torch.cat([input_ids, next_token_id], dim=-1)

    next_word = tokenizer.decode(next_token_id[0])
    print(f"Step {step+1}: '{next_word}'")

print(f"\nFinal: {tokenizer.decode(input_ids[0], skip_special_tokens=True)}")


Starting text: 'Yesterday I learned how to skateboard but I injured my knee in the process'

Generating word by word...

Step 1: '.'
Step 2: '
'
Step 3: 'I'
Step 4: ' had'
Step 5: ' a'
Step 6: ' few'
Step 7: ' days'
Step 8: ' off'
Step 9: ' and'
Step 10: ' it'
Step 11: ' wasn'
Step 12: ''t'
Step 13: ' until'
Step 14: ' this'
Step 15: ' morning'
Step 16: ' that'
Step 17: ' after'
Step 18: ' getting'
Step 19: ' back'
Step 20: ' on'
Step 21: ' track'
Step 22: ','
Step 23: ' we'
Step 24: ' started'
Step 25: ' working'
Step 26: ' together'
Step 27: ' with'
Step 28: ' our'
Step 29: ' friends'
Step 30: ' from'

Final: Yesterday I learned how to skateboard but I injured my knee in the process.
I had a few days off and it wasn't until this morning that after getting back on track, we started working together with our friends from
